In [34]:
from wikiparser_utils import WikiXMLDump, WikiPage
import os
import nltk
import wikitextparser as wtp
import json
import numpy as np
from tqdm import tqdm
from difflib import Differ 
from utils.difflibparser import DifflibParser, DiffCode
from transformers import T5TokenizerFast, T5ForConditionalGeneration
import shutil
import re

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from io import StringIO, BytesIO
from PyPDF2 import PdfReader, PdfFileReader
from collections import Counter
import html2text
import os
import time
from os import listdir
from os.path import isfile, join

from selenium.webdriver.chrome.options import Options
import mwxml
import wikitextparser as wtp

In [2]:
DOCS_DIR = 'data/documents_new'
DOCS_MAPPER_DIR = 'data/documents_mapper_new'

if not os.path.exists(DOCS_MAPPER_DIR):
    os.makedirs(DOCS_MAPPER_DIR)
    
if not os.path.exists(DOCS_DIR):
    os.makedirs(DOCS_DIR)


In [3]:
def filter_page(page_name):
    return bool(re.search('[a-zA-Z]', page_name))

def filter_comment(comment_text, user_name):
    com_text = comment_text.strip()
    if 'bot' in com_text or 'bot' in user_name:
        return False
    
    if com_text[-2:] == '*/':
        return False
    return True

In [4]:
MIN_CHANGE_SYMB_LEN = 5
MAX_CHANGE_SYMB_LEN = 300
MAX_PAGE_SYMB_LEN = 15000
MAX_ABSTRACT_LEN = 800
MIN_ABSTRACT_LEN = 10
MIN_COMMENT_LEN = 5
abstarct_tokenizer = lambda x: x

In [5]:
def find_nearest(l, r, arr):
    l_ans, r_ans = -1, -1
    for sent_idx, (l_arr, r_arr) in enumerate(zip(arr, arr[1:])):
        if l_arr <= l < r_arr:
            l_ans = sent_idx
        if l_arr <= r < r_arr:
            r_ans = sent_idx
    if l_ans == -1:
        l_ans = len(arr) - 1
    if r_ans == -1:
        r_ans = len(arr) - 1
    return l_ans, r_ans

In [15]:
def clean_text(text):
    text = text.replace('=====', '==').replace('====', '==').replace('===', '==')
    text = re.sub('\[\[File.*?]]', '', text, count=0, flags=0)
    text = re.sub('\[\[Category:.*?]]', '', text, count=0, flags=0)
    text = re.sub('\[\[category:.*?]]', '', text, count=0, flags=0)
    text = wtp.remove_markup(text)
    text = text.replace('\t', '').replace('\n\n\n', '\n\n').replace('\n\n\n', '\n\n')
    text = text.replace('\n\n*', ', ').replace('\n\n*', ', ')
    return text
    
def clean_section_text(text):
    text = re.sub('==.*?==+', '', text, count=0, flags=0)
    return text.strip()

def text2sentences(text, sent_tokenizer=nltk.sent_tokenize):
    idxs_arr = []
    sents = sent_tokenizer(text)
    cur_str = text[:]
    cur_skip = 0
    idxs2sent = {}
    for sent in sents:
        match_idx = cur_str.find(sent)
        start_idx = match_idx + cur_skip
        idxs_arr.append(start_idx)
        finish_idx = match_idx + cur_skip + len(sent) - 1
        idxs2sent[(start_idx, finish_idx)] = sent
        if finish_idx + 1 < len(cur_str):
            cur_skip = finish_idx + 1
            cur_str = cur_str[match_idx + len(sent):]
    return idxs2sent, np.array(sents), idxs_arr

def extract_important_sections(text):
    parsed_text = wtp.parse(text)
    section_titles, section_texts = [], []
    for sec in parsed_text.sections:
        if not sec.title:
            #for par in sec.string.split('\n\n'):
            section_titles.append(sec.title)
            section_texts.append(clean_section_text(sec.string))
            continue
        if 'external links' in sec.title.lower():
            continue
        if 'references' in sec.title.lower():
            continue
        if 'notes' in sec.title.lower():
            continue
        if 'see also' in sec.title.lower():
            continue
        
        #for par in sec.string.split('\n\n'):
        section_titles.append(sec.title)
        section_texts.append(clean_section_text(sec.string))
    return section_titles, section_texts

def get_diff_num(prev_sections_texts, new_sections_texts):
    prev_set = set(enumerate(prev_sections_texts))
    new_set = set(enumerate(new_sections_texts))
    new_diff = new_set - prev_set
    prev_diff = prev_set - new_set
    if len(new_diff) > 0 and len(prev_diff) > 0:
        print('\n\n#############################################')
        print('WAS:\n')
        print(new_diff)
        print('\n-------------------------------------------\nNOW:\n')
        print(prev_diff)
        print('\n')
        return True
    return False

def get_diff_num2(prev_sections_texts, new_sections_texts):
    differ_obj = Differ()
    dif_result = list(DifflibParser(prev_sections_texts, new_sections_texts))
    result = []
    result_idxs = []
    old_text, new_text, last_diff_id = [], [], -1000
    for dif_id, dif_line in enumerate(dif_result):
        if dif_line['code'] != DiffCode.SIMILAR:
            if np.abs(dif_id - last_diff_id) > 0:
                result.append(dif_line)
                result_idxs.append(dif_id)
                last_diff_id = dif_id
    return result_idxs, result    

def get_changes(diffs):
    all_changes = []
    all_changes_sents = []
    for diff_id, diff_obj in enumerate(diffs):
        if diff_obj['code'] == DiffCode.RIGHTONLY:
            if len(abstarct_tokenizer(diff_obj['line'])) > MAX_ABSTRACT_LEN:
                continue
            if len(abstarct_tokenizer(diff_obj['line'])) < MIN_ABSTRACT_LEN:
                continue
            all_changes.append(([diff_obj['line']], 'r'))
            _, sents, _ = text2sentences(diff_obj['line'])
            all_changes_sents.append(sents)
            
        elif diff_obj['code'] == DiffCode.LEFTONLY:
            if len(abstarct_tokenizer(diff_obj['line'])) > MAX_ABSTRACT_LEN:
                continue
            if len(abstarct_tokenizer(diff_obj['line'])) < MIN_ABSTRACT_LEN:
                continue
            all_changes.append(([diff_obj['line']], 'l'))
            _, sents, _ = text2sentences(diff_obj['line'])
            all_changes_sents.append(sents)
            
        elif diff_obj['code'] == DiffCode.CHANGED:
            if len(abstarct_tokenizer(diff_obj['line'])) > MAX_ABSTRACT_LEN:
                continue
            if len(abstarct_tokenizer(diff_obj['newline'])) > MAX_ABSTRACT_LEN:
                continue
            idxs2sent, sents, idxs_arr = text2sentences(diff_obj['newline'])
            all_changes_sents = []
            r_change = diff_obj['rightchanges']
            cur_ch = -10
            prev_ch = -10
            all_r_changes = []
            changed_sents = []
            for ch in r_change:
                if prev_ch < 0:
                    prev_ch = ch
                    cur_ch = ch
                if np.abs(ch - cur_ch) > 1:
                    new_change = diff_obj['newline'][prev_ch:cur_ch+1]
                    if new_change.strip() != '' and len(new_change.strip()) > MIN_CHANGE_SYMB_LEN:
                        all_r_changes.append(new_change)
                        sents_idxs_l, sents_idxs_r = find_nearest(prev_ch, cur_ch+1, idxs_arr)
                        changed_sents += list(range(sents_idxs_l, sents_idxs_r+1))
                    prev_ch = ch
                cur_ch = ch
            new_change = diff_obj['newline'][prev_ch:cur_ch+1]
            if new_change.strip() != '' and len(new_change.strip()) > MIN_CHANGE_SYMB_LEN:
                all_r_changes.append(new_change)
                sents_idxs_l, sents_idxs_r = find_nearest(prev_ch, cur_ch+1, idxs_arr)
                changed_sents += list(range(sents_idxs_l, sents_idxs_r+1))
            all_changes.append((all_r_changes, 'c'))
            changed_sents = sorted(list(set(changed_sents)))
            all_changes_sents.append(sents[changed_sents])
    return all_changes, all_changes_sents

In [7]:
import datetime
print(datetime.datetime.now())

2022-11-26 00:46:52.478493


In [10]:
DOC_COUNTER = 0
W2DC = Counter()
dump = mwxml.Dump.from_file(open('data/history6_last.xml', encoding="utf-8"))
pbar = tqdm(position=0, leave=True)
for page in dump:
    revisions = []
    for rev in page:
        revisions.append(rev)
    last_rev = revisions[-1]
    last_rev_text = ''
    if last_rev.text:
        last_rev_text = clean_text(last_rev.text).lower()
    
    tokens = list(set(nltk.word_tokenize(last_rev_text)))
    W2DC.update(tokens)
    DOC_COUNTER += 1
    pbar.update(1)

180it [00:15, 11.44it/s]
7817it [06:05, 12.44it/s]

In [11]:
W2DC.most_common(10)

[(',', 4425),
 ('the', 4358),
 ('of', 4316),
 ('.', 4225),
 (')', 4190),
 ('(', 4188),
 ('a', 4026),
 ('and', 4012),
 ('to', 3958),
 ('in', 3904)]

In [13]:
DOC_COUNTER

7817

In [31]:
import datetime
import time
start_time = datetime.datetime.now()
print(start_time)

2022-11-26 13:50:03.105809


In [36]:
options = Options()
options.headless = True
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')
driver_main = webdriver.Chrome(options=options)
driver_main.set_page_load_timeout(180)

# открываем сайт с таблицой об игроках с players_ranks[0] до players_ranks[1] 
driver_main.get(f'https://duckduckgo.com/')
    
# временами спим, чтобы имитировать поведение пользователя
time.sleep(2)

In [40]:
def find_docs_by_query(query, driver_main):
    get_url = driver_main.current_url
    version = 'lite' 
    if 'lite.duckduckgo' in get_url:
        version = 'lite'
    elif 'html.duckduckgo' in get_url:
        version = 'html'
    else:
        version = 'norm'
    
    if version == 'norm':
        search_input_pattern = 'js-search-input' 
        submit_pattern = 'search__button' 
        result_snippet_pattern = 'result__snippet' 
        next_button = "//input[@value='Next']" 
    elif version == 'html':
        search_input_pattern = 'search__input' 
        submit_pattern = 'search__button' 
        result_snippet_pattern = 'result__snippet' 
        next_button = "//input[@value='Next']" 
    else:
        search_input_pattern = 'query'
        submit_pattern = 'submit'
        result_snippet_pattern = 'result-snippet' 
        next_button = "//input[@value='Next Page >']"
    
    search_form = driver_main.find_element(By.CLASS_NAME, search_input_pattern)
    search_form.send_keys(query)
    driver_main.find_element(By.CLASS_NAME, submit_pattern).click()
    
    snipets_texts = set()
    for i in range(2):
        snips = driver_main.find_elements(By.CLASS_NAME, result_snippet_pattern)
        snips = snips if snips else []
        for snip in snips:
            snipets_texts.add(snip.text)
    
        try:
            driver_main.find_element(By.XPATH, next_button).click()
        except NoSuchElementException:
            pass
        except:
            print('BAD NEXT TRY', query)
        
    
    search_form = driver_main.find_element(By.CLASS_NAME, search_input_pattern)
    search_form.clear()
    return list(snipets_texts)

In [48]:
def count_doc_score(doc_text):
    doc_toks = nltk.word_tokenize(doc_text.lower())
    toks_counter = Counter(doc_toks)
    score = 0.0
    for tok, tok_count in toks_counter.items():
        if W2DC[tok] > 0:
            score += tok_count * np.log(DOC_COUNTER / W2DC[tok])
    return score

In [49]:
search_result = find_docs_by_query('Henry Blogg Legacy', driver_main)
search_scored_result = [(st, count_doc_score(st)) for st in search_result]

In [50]:
sorted_search_scored_result = sorted(search_scored_result, key=lambda x: -x[1])
sorted_search_scored_result[:3]

[('In 1948 Millie Walton was renamed Henry Blogg in homage to Cromer\'s famous lifeboat man Henry Blogg who was the Coxswain. Henry Blogg made his last voyage on the Millie Walton under the new coxswain Henry "Shrimp" Davies on 4 September 1948 at the age of 71. The call was to the rescue of the steam trawler Balmoral and 11 lives were saved. Henry Blogg retired after 53 years service with the service and he was the holder of the most awards by the RNLI. Notable rescues',
  290.9407857667947),
 ('Henry George Blogg GC BEM was a lifeboatman from Cromer on the north coast of Norfolk, England, and the most decorated in Royal National Lifeboat Institution history. Blogg of the Cromer Lifeboat Station is referred to as "the greatest of the lifeboatmen". From the rescue of the crew of the Pyrin and then of half of the crew of the Fernebo in 1917, through to his near drowning in the course of the rescue of survivors of the wreck of the SS English Trader in 1941, he was awarded the gold medal 

In [52]:
file = open('dif0.txt', 'w')
counter = 0
dump = mwxml.Dump.from_file(open('data/history6_last.xml', encoding="utf-8"))
pbar = tqdm(position=0, leave=True)
for page in dump:
    
    if not filter_page(page.title):
        # pbar.update(1)
        continue
    revisions = []
    for rev in page:
        revisions.append(rev)
    
    if len(revisions) < 2: 
        # pbar.update(1)
        continue
    good_revisions = []
    
    last_added = len(revisions)
    for cur_rev_id in range(len(revisions) - 1, 1, -1):
        if cur_rev_id >= last_added:
            continue
        cur_rev = revisions[cur_rev_id]
        if cur_rev.text:
            cur_rev_text = cur_rev.text # clean_text(cur_rev.text)
        else:
            cur_rev_text = ''
        
        for new_rev_id in range(cur_rev_id, 0, -1):
            new_rev = revisions[new_rev_id]
            
            if new_rev.text:
                new_rev_text = new_rev.text # clean_text(new_rev.text)
            else:
                new_rev_text = ''
            
            if cur_rev_text == new_rev_text:
                last_added = new_rev_id
        
        add_rev = revisions[last_added]
        user = ''
        if add_rev.user:
            if add_rev.user.text:
                user = add_rev.user.text.lower()
        revision_dict = {
            'text': cur_rev_text,
            'comment': add_rev.comment,
            'id': add_rev.id,
            'page_name': page.title,
            'user_name': user
        }
        good_revisions.append(revision_dict)
        # pbar.update(1)
    good_revisions = good_revisions[::-1]
              
    for prev_rev, new_rev in zip(good_revisions[:], good_revisions[1:]):
        comment = new_rev['comment']
        if comment and len(comment.strip()) > MIN_COMMENT_LEN:
            if filter_comment(comment, new_rev['user_name']):
                if np.abs(len(new_rev['text']) - len(prev_rev['text'])) > MIN_CHANGE_SYMB_LEN:
                    if np.abs(len(new_rev['text']) - len(prev_rev['text'])) < MAX_CHANGE_SYMB_LEN:
                        if np.abs(len(new_rev['text'])) < MAX_PAGE_SYMB_LEN:
                            prev_section_titles, prev_section_texts = extract_important_sections(clean_text(prev_rev['text']))
                            new_section_titles, new_section_texts = extract_important_sections(clean_text(new_rev['text']))

                            r_idx, r = get_diff_num2(prev_section_texts, new_section_texts)
                            if len(r) == 1:
                                section_name = ''
                                try:
                                    section_name_t = new_section_titles[r_idx[0]]
                                    if section_name_t:
                                        section_name = section_name_t
                                except:
                                    pass
                                
                                
                                all_changes_r, all_changes_sents_r = get_changes(r)
                                if len(all_changes_sents_r) > 0 and len(all_changes_sents_r[0]) > 0:
                                    changes_text, changes_sents = get_changes(r)
                                    print('Page name:\t', new_rev['page_name'])
                                    print('Diff section name:\t', section_name)
                                    print('Diff comment:\t', comment)
                                    
                                    print('###############################################\n')
                                    if 'newline' in r[0]:
                                        print('WAS:\n')
                                        print(r[0]['line'])
                                        print('\nNEW:\n')
                                        print(r[0]['newline'])
                                    else:
                                        print('NEW:\n')
                                        print(r[0]['line'])
                                    print('###############################################\n')
                                    
                                    diff_text = ' '.join(changes_sents[0]).lower()
                                    diff_tokens = list(set(nltk.word_tokenize(diff_text)))
                                    ans_tokens = []
                                    for tok in diff_tokens:
                                        if tok in W2DC:
                                            tok_df = W2DC[tok] / DOC_COUNTER
                                        else:
                                            tok_df = 0
                                        
                                        if tok_df < 0.001:
                                            ans_tokens.append(tok)
                                    
                                    changed_text = [ctxt for ctxt in all_changes_r[0][0]]
                                    changed_text_full = ' '.join(changed_text)
                                    
                                    good_toks_str = ' '.join(ans_tokens)
                                    print('Change good toks:\t', good_toks_str)
                                    ts = new_rev['page_name'] + ' ' + section_name
                                    fq = ts.strip() + ' ' + good_toks_str
                                    print('Final search query:\t', fq)
                                    fq_2 = ts.strip() + ' ' + changed_text_full
                                    print('Final search query res2:\t', fq_2)
                                    
                                    search_result = find_docs_by_query(fq_2, driver_main)
                                    search_scored_result = [(st, count_doc_score(st)) for st in search_result]
                                    sorted_search_scored_result = sorted(search_scored_result, key=lambda x: -x[1])
                                    
                                    print('\n###############################\n\n')
                                    for doc_id, (doc_text, doc_score) in enumerate(sorted_search_scored_result[:3]):
                                        print(f'DOC {doc_id}\t----------\t{doc_score}\n{doc_text}\n\n')
                                    print('\n\n\n------------------------------------------\n\n')
                                     
                                    '''
                                    file.write(comment)
                                    file.write('\t---------\t')
                                    file.write(new_rev['user_name'])
                                    file.write('\n\n\n')

                                    file.write(str(r))
                                    file.write('\n####################################\n')
                                    file.write(str(get_changes(r)))
                                    file.write('\n\n___________________________________\n\n')
                                    #for tt, ttt in zip(new_section_titles, new_section_texts):
                                    #    print(tt, '\t----------\t', ttt)
                                    counter += 1
                                    '''
                                    counter += 1
        pbar.update(1)
    if counter > 50:
        break
file.close()
                                  
                                  

4075it [3:52:22,  3.42s/it]
3it [00:00, 25.78it/s]

Page name:	 Henry Blogg
Diff section name:	 
Diff comment:	 /* Further reading */ cat + link
###############################################

NEW:

"Henry Blogg, the Greatest of the Lifeboatmen", Jolly, C., Pub: Poppyland Publishing, new edition 2002, ISBN 0946148597
###############################################

Change good toks:	 lifeboatmen poppyland blogg 0946148597
Final search query:	 Henry Blogg lifeboatmen poppyland blogg 0946148597
Final search query res2:	 Henry Blogg "Henry Blogg, the Greatest of the Lifeboatmen", Jolly, C., Pub: Poppyland Publishing, new edition 2002, ISBN 0946148597


19it [00:03,  7.84it/s]


###############################


DOC 0	----------	308.3269388466724
Storia. Il Gallois era una nave mercantile a vapore costruita nel 1917 da Wood, Skinner & Company Ltd., Newcastle sul Tyne, Inghilterra. Lei aveva 2.684tonnellate di registro lorde (GRT) e 321 piedi (98 m) di lunghezza. Il suo numero di cantiere era No: 197. Era stata ordinata dalla Burnett Steam Ship Co. Ltd., (Burnett & Co) di Newcastle sul Tyne.Il suo nome originale era SS Tynemouth.


DOC 1	----------	245.8247046290446
RNLI Henry Blogg Museum RNLI Henry Blogg Museum RNLI Henry Blogg Museum RNLI Henry Blogg Museum Phone: +44 1263 511294 Hours: Sunday: 10am - 5pm: Monday: Closed: Tuesday: 10am - 5pm: Wednesday: 10am - 5pm: Thursday: 10am - 5pm: Friday: 10am - 5pm: Saturday:


DOC 2	----------	236.47692783032394
RNLB H F Bailey II (ON 714) was the third lifeboat stationed at Cromer to bear this name, but the first of the four to bear a number in its name. This lifeboat was sent to replace H F Bailey (ON 694). Coxswa

60it [00:03, 32.17it/s]

...

7034it [03:20, 12.92it/s]

Page name:	 List of t'ai chi ch'uan forms
Diff section name:	 Weapon forms
Diff comment:	 /* See also */ {{wikiquote|Tai chi chuan}}
###############################################

WAS:

* 13 - Dong Yue (East Mountain) Combined Sword
* 16 - Yang Standardized Sword
* 16 - Yang/Combined Standardized Spear
* 13 - Yang Broadsword (Dao)
* 18 - Chen 'Health' Standardized Sword
* 23 - Chen Broadsword
* 27 - 5 Section solo Taijijian]] 
* 32 - Yang/Combined  Sword
* 36 - Chen Shi (Chen Style) Xinyi Hun Yuan broadsword
* 40 - Chen Broadsword
* 48 - Chen Shi (Chen Style) Xinyi Hun Yuan sword
* 49 - Chen Sword 
* 54 - Yang Sword
* 56 - Fu style tai chi seven star sword
* 62 - Chen Single Sword
* 64 - 5 Section 2 person Taijijian
* 92 - Wudang Single Sword

NEW:

* 13 - Dong Yue (East Mountain) Combined Sword
* 16 - Yang Standardized Sword
* 16 - Yang/Combined Standardized Spear
* 13 - Yang Broadsword (Dao)
* 18 - Chen 'Health' Standardized Sword
* 23 - Chen Broadsword
* 27 - 5 Section solo Taijij

7090it [03:26, 12.52it/s]


###############################


DOC 0	----------	305.3146268950655
Weapon Forms. 13 - Dong Yue (East Mountain) Combined Sword; 13 - Wu Jianquan Spear (3.5 meter cavalry lance) 13 - Beijing Northern Wu Style Tai Ji Shi San Dao. 13 - Posture Poem Yang family tai chi chuan Saber Form; 13 - Yang family Broadsword (Dao) 16 - Yang Standardized Sword; 16 - Yang/Combined Standardized Spear; 18 - Chen 'Health ...


DOC 1	----------	275.5739611991818
Wu Yu Xiang leaned the Art from Yang Lu Chan and a variation of the original Chen form from Chen Ching Ping (who taught the 'small frame' version of Chen Tai Ji Quan) and created the Wu style. A man named Hao Wei Zhen, learned the Wu style from Wu Yu Xiang's nephew and taught the style to Sun Lu Tang, who in turn created the Sun style (Sun was ...


DOC 2	----------	255.59926718366646
Styl Chen (陳氏) Styl Yang (楊家) Styl Wu Wu Ch'uan-yu i Wu Chien-ch'uan (吳家) Wu lub Wu/Hao w stylu Wu Yu-hsianga (武家) W stylu słońca (孫家) Styl Yangjia Michuan (楊家秘傳) S

7168it [03:37, 23.38it/s]

In [16]:
counter

12132

In [17]:
finish_time = datetime.datetime.now()
print(start_time)
print(finish_time)

2022-11-25 23:10:35.610584
2022-11-25 23:40:34.961865


615381it [30:10, 1064.54it/s]

In [ ]:
print(start_time)
print(finish_time)

In [ ]:
counter

In [54]:
45 * 775

34875

In [7]:
print(page_name)
json.dump(page_docs_mapper_id2link, open(f'{DOCS_MAPPER_DIR}/{page_name}/id2link.json', "w"))
json.dump(page_docs_mapper_link2id, open(f'{DOCS_MAPPER_DIR}/{page_name}/link2id.json', "w"))

Darkwood Dub_2935508
